In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 kB 796.1 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.11.1-py3-none-any.whl (227 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp310-cp310-linux_x86_64.whl size=4199774 sha256=2d9dbdf256d6238443725fc5bd0d3eeb6c2e5b53d9e5c66e1b198d12e3e2eca1
  Stored in directory: /root/.cache/pip/wheels/a5/13/75/f811c84a8ab36eedbaef977a6a58a98990e8e0f1967f98f394
Successfully built fasttext


In [3]:
seed_value = 777

In [4]:
import fasttext

import pandas as pd
import numpy as np
import random
import statsmodels as sm
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)


import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils.class_weight import compute_class_weight


from sklearn.metrics import accuracy_score

from lightgbm import LGBMClassifier

import gensim
from gensim.models import Word2Vec
from gensim.models import FastText

import joblib

import tensorflow as tf
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, LayerNormalization
from tensorflow.keras.layers import MultiHeadAttention, Lambda
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GlobalAveragePooling1D
from tensorflow.keras import Input, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy, CategoricalCrossentropy

from tensorflow.keras.models import save_model
from tensorflow.keras.models import load_model

tf.random.set_seed(seed_value)
np.random.seed(seed_value)
random.seed(seed_value)

In [ ]:
# column의 각 행이 동일한 데이터로 되어있는지 확인하는 함수
# ex) a,b,c로 되어있는 행에서 a=b=c인지 확인, 다르면 list에 append

def different_value_count(df, column):
    different_label = []

    for i in range(df.shape[0]):
        type_value = df.loc[df.index[i], column]
        if len(set(type_value)) > 1:
            different_label.append(df.index[i])

    print(len(different_label))

In [ ]:
# column의 각 행이 동일한 데이터로 되어있는지 확인하는 함수
# ex) a,b,c로 되어있는 행에서 a=b=c인지 확인

def different_value_df(df, column):
    different_index = []

    for i in range(df.shape[0]):
        type_value = df.loc[df.index[i], column]
        if len(set(type_value)) > 1:
            different_index.append(df.index[i])

    different_label = pd.DataFrame(different_index, columns = ['ticketno'])
    different_label = pd.merge(different_label, df[column], how = 'inner', on = 'ticketno')

    return different_label

In [ ]:
# root_cause_type의 각 행의 값이 동일하면 맨 처음 값으로 변경되는 함수
# ex) a=b=c ==> a로 변경

def replace_root_cause_type(row):
    if 'PowerFail' in row:
        return 'PowerFail'
    elif 'UnitFail' in row:
        return 'UnitFail'
    elif 'LinkCut' in row:
        return 'LinkCut'
    return row

In [ ]:
def unpack_list(lst):
    """
    list를 str로 변환하는 함수

    Args:
        lst (list): str로 변환하고 싶은 list

    Returns:
        str로 변환된 list의 요소들을 ','로 연결시킨 문자열
    """

    return ','.join(str(x) for x in lst)

# **Data**

In [ ]:
# Raw data

train_org = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/백지짱_분야2/Submit/data/Q2_train.csv')
test_org = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/백지짱_분야2/Submit/data/Q2_test.csv')
label_sample_org = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/백지짱_분야2/Submit/data/Q2_label_sample.csv')

In [ ]:
train_change_msg = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/백지짱_분야2/Submit/data/train_map_3.csv')
test_change_msg = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/백지짱_분야2/Submit/data/test_map_3.csv')

# **msg sorting with sysname & alarmmsg**

## **train, test divide**

In [ ]:
# 필요 컬럼들만 모아 새로운 df 생성
# ticketno, alarmno, alarmtime 순으로 정렬 (asc)
col = ['ticketno', 'alarmno', 'alarmtime', 'alarmlevel', 'alarmmsg_original', 'site', 'root_cause_type']

train_divide = train_change_msg[col].sort_values(['alarmtime', 'site'])
train_divide = train_divide.drop('alarmtime', axis = 1)
train_divide.head()

,ticketno,alarmno,alarmlevel,alarmmsg_original,site,root_cause_type
0,21122633.0,1669820428245,5,ETHERNET-ERROR,ACEN,LinkCut
1,21122633.0,1669821318728,5,ETHERNET-ERROR,ACEN,LinkCut
2,21122633.0,1669822214832,5,ETHERNET-ERROR,ACEN,LinkCut
3,21122633.0,1669823114128,5,ETHERNET-ERROR,ACEN,LinkCut
4,21122633.0,1669824028082,5,ETHERNET-ERROR,ACEN,LinkCut


In [ ]:
train_new = train_divide.groupby(['ticketno', 'site'])['alarmmsg_original', 'root_cause_type'].agg(lambda x: ','.join(sorted(x)))\
                                                                                              .reset_index()
train_new.head()

<ipython-input-266-b46212ac6eb8>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  train_new = train_divide.groupby(['ticketno', 'site'])['alarmmsg_original', 'root_cause_type'].agg(lambda x: ','.join(sorted(x)))\


,ticketno,site,alarmmsg_original,root_cause_type
0,14753084.0,ABNE,"ETHERNET-LINK-FAIL,ETHERNET-LINK-FAIL,ETHERNET...","PowerFail,PowerFail,PowerFail,PowerFail,PowerF..."
1,14771766.0,ABLB,"ETHERNET-LINK-FAIL,ETHERNET-LINK-FAIL,POWER-SU...","PowerFail,PowerFail,PowerFail,PowerFail"
2,14777089.0,ABLZ,"ETHERNET-LINK-FAIL,ETHERNET-LINK-FAIL,ETHERNET...","PowerFail,PowerFail,PowerFail,PowerFail,PowerFail"
3,14790052.0,ACRX,"POWER-SUPPLY-UNIT-FAIL,POWER-SUPPLY-UNIT-FAIL","PowerFail,PowerFail"
4,14879922.0,ABNE,"ETHERNET-LINK-FAIL,ETHERNET-LINK-FAIL,ETHERNET...","PowerFail,PowerFail,PowerFail,PowerFail,PowerF..."


In [ ]:
train_new = train_new.groupby(['ticketno'])['alarmmsg_original', 'root_cause_type'].agg(lambda x: ','.join(sorted(x))).reset_index()
train_new.head()

<ipython-input-267-452315a9e290>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  train_new = train_new.groupby(['ticketno'])['alarmmsg_original', 'root_cause_type'].agg(lambda x: ','.join(sorted(x))).reset_index()


,ticketno,alarmmsg_original,root_cause_type
0,14753084.0,"ETHERNET-LINK-FAIL,ETHERNET-LINK-FAIL,ETHERNET...","PowerFail,PowerFail,PowerFail,PowerFail,PowerF..."
1,14771766.0,"ETHERNET-LINK-FAIL,ETHERNET-LINK-FAIL,POWER-SU...","PowerFail,PowerFail,PowerFail,PowerFail"
2,14777089.0,"ETHERNET-LINK-FAIL,ETHERNET-LINK-FAIL,ETHERNET...","PowerFail,PowerFail,PowerFail,PowerFail,PowerFail"
3,14790052.0,"POWER-SUPPLY-UNIT-FAIL,POWER-SUPPLY-UNIT-FAIL","PowerFail,PowerFail"
4,14879922.0,"ETHERNET-LINK-FAIL,ETHERNET-LINK-FAIL,ETHERNET...","PowerFail,PowerFail,PowerFail,PowerFail,PowerF..."


In [ ]:
train_new['alarmmsg_original'] = train_new['alarmmsg_original'].apply(lambda x: ','.join(pd.unique(x.split(','))))
train_new.head()

,ticketno,alarmmsg_original,root_cause_type
0,14753084.0,"ETHERNET-LINK-FAIL,ETHERNET-NO-RECEIVE-TRAFFIC...","PowerFail,PowerFail,PowerFail,PowerFail,PowerF..."
1,14771766.0,"ETHERNET-LINK-FAIL,POWER-SUPPLY-UNIT-FAIL","PowerFail,PowerFail,PowerFail,PowerFail"
2,14777089.0,"ETHERNET-LINK-FAIL,POWER-SUPPLY-UNIT-FAIL","PowerFail,PowerFail,PowerFail,PowerFail,PowerFail"
3,14790052.0,POWER-SUPPLY-UNIT-FAIL,"PowerFail,PowerFail"
4,14879922.0,"ETHERNET-LINK-FAIL,ETHERNET-NO-RECEIVE-TRAFFIC...","PowerFail,PowerFail,PowerFail,PowerFail,PowerF..."


In [ ]:
train_new.shape

(1114, 3)

In [ ]:
# 필요 컬럼들만 모아 새로운 df 생성
# (asc)
col_test = ['ticketno', 'alarmno', 'alarmtime', 'alarmlevel', 'alarmmsg_original', 'site']

test_divide = test_change_msg[col_test].sort_values(['alarmtime'])
test_divide.head()

,ticketno,alarmno,alarmtime,alarmlevel,alarmmsg_original,site
0,21812391.0,1671894138838,2022-12-25 00:02:16+09:00,5,LOSS-OF-SIGNAL,AEAQ
1,21775988.0,1671894172511,2022-12-25 00:02:51+09:00,5,LOSS-OF-SIGNAL,ADZW
2,21792259.0,1671894204682,2022-12-25 00:03:22+09:00,4,ALARM-RECEIVE-POWER-HIGH,AECE
3,21812412.0,1671894215702,2022-12-25 00:03:33+09:00,5,BATTERY-ENVIRONMENT-FAIL,ACCN
4,21812417.0,1671894220812,2022-12-25 00:03:39+09:00,5,LOSS-OF-SIGNAL,AEAQ


In [ ]:
test_new = test_divide.groupby(['ticketno', 'site'])['alarmmsg_original'].agg(lambda x: ','.join(sorted(x))).reset_index()
test_new.head()

,ticketno,site,alarmmsg_original
0,15238899.0,ABMY,"ETHERNET-LINK-FAIL,ETHERNET-LINK-FAIL,POWER-SU..."
1,15712444.0,AEOK,"POWER-SUPPLY-UNIT-FAIL,POWER-SUPPLY-UNIT-FAIL"
2,15723187.0,ACTL,"ETHERNET-NO-RECEIVE-TRAFFIC,POWER-SUPPLY-UNIT-..."
3,15737103.0,ACQL,"ETHERNET-LINK-FAIL,ETHERNET-LINK-FAIL,POWER-SU..."
4,15737132.0,ACQL,"ETHERNET-LINK-FAIL,ETHERNET-LINK-FAIL,POWER-SU..."


In [ ]:
test_new = test_new.groupby(['ticketno'])['alarmmsg_original'].agg(lambda x: ','.join(sorted(x))).reset_index()
test_new.head()

,ticketno,alarmmsg_original
0,15238899.0,"ETHERNET-LINK-FAIL,ETHERNET-LINK-FAIL,POWER-SU..."
1,15712444.0,"POWER-SUPPLY-UNIT-FAIL,POWER-SUPPLY-UNIT-FAIL"
2,15723187.0,"ETHERNET-NO-RECEIVE-TRAFFIC,POWER-SUPPLY-UNIT-..."
3,15737103.0,"ETHERNET-LINK-FAIL,ETHERNET-LINK-FAIL,POWER-SU..."
4,15737132.0,"ETHERNET-LINK-FAIL,ETHERNET-LINK-FAIL,POWER-SU..."


In [ ]:
test_new['alarmmsg_original'] = test_new['alarmmsg_original'].apply(lambda x: ','.join(pd.unique(x.split(','))))
test_new.head()

,ticketno,alarmmsg_original
0,15238899.0,"ETHERNET-LINK-FAIL,POWER-SUPPLY-UNIT-FAIL"
1,15712444.0,POWER-SUPPLY-UNIT-FAIL
2,15723187.0,"ETHERNET-NO-RECEIVE-TRAFFIC,POWER-SUPPLY-UNIT-..."
3,15737103.0,"ETHERNET-LINK-FAIL,POWER-SUPPLY-UNIT-FAIL"
4,15737132.0,"ETHERNET-LINK-FAIL,POWER-SUPPLY-UNIT-FAIL"


In [ ]:
test_new.shape

(4327, 2)

## **train**

In [ ]:
# ticketno를 그룹화하여 level과 msg 리스트로 만들기

train_site = pd.DataFrame(train_new.groupby(['ticketno'])[['alarmmsg_original', 'root_cause_type']].agg(list))
train_site.head()

,alarmmsg_original,root_cause_type
ticketno,,
14753084.0,"[ETHERNET-LINK-FAIL,ETHERNET-NO-RECEIVE-TRAFFI...","[PowerFail,PowerFail,PowerFail,PowerFail,Power..."
14771766.0,"[ETHERNET-LINK-FAIL,POWER-SUPPLY-UNIT-FAIL]","[PowerFail,PowerFail,PowerFail,PowerFail]"
14777089.0,"[ETHERNET-LINK-FAIL,POWER-SUPPLY-UNIT-FAIL]","[PowerFail,PowerFail,PowerFail,PowerFail,Power..."
14790052.0,[POWER-SUPPLY-UNIT-FAIL],"[PowerFail,PowerFail]"
14879922.0,"[ETHERNET-LINK-FAIL,ETHERNET-NO-RECEIVE-TRAFFI...","[PowerFail,PowerFail,PowerFail,PowerFail,Power..."


In [ ]:
max_count_ticketno = max(train_site['alarmmsg_original'].apply(len))
max_count_ticketno

1

In [ ]:
# root_cause_type의 변경 값이 없는지 확인
different_value_count(train_site, 'root_cause_type')

0


In [ ]:
# alarmmsg_original 변경 값이 없는지 확인
different_value_count(train_site, 'alarmmsg_original')

0


In [ ]:
# map(lambda)를 사용하여 리스트 풀기
train_site = pd.DataFrame(train_site.applymap(lambda x: unpack_list(x)))

print(len(train_site))
train_site.head(10)

1114


,alarmmsg_original,root_cause_type
ticketno,,
14753084.0,"ETHERNET-LINK-FAIL,ETHERNET-NO-RECEIVE-TRAFFIC...","PowerFail,PowerFail,PowerFail,PowerFail,PowerF..."
14771766.0,"ETHERNET-LINK-FAIL,POWER-SUPPLY-UNIT-FAIL","PowerFail,PowerFail,PowerFail,PowerFail"
14777089.0,"ETHERNET-LINK-FAIL,POWER-SUPPLY-UNIT-FAIL","PowerFail,PowerFail,PowerFail,PowerFail,PowerFail"
14790052.0,POWER-SUPPLY-UNIT-FAIL,"PowerFail,PowerFail"
14879922.0,"ETHERNET-LINK-FAIL,ETHERNET-NO-RECEIVE-TRAFFIC...","PowerFail,PowerFail,PowerFail,PowerFail,PowerF..."
14901137.0,"ETHERNET-LINK-FAIL,ETHERNET-NO-RECEIVE-TRAFFIC...","PowerFail,PowerFail,PowerFail,PowerFail,PowerF..."
14919180.0,"ETHERNET-LINK-FAIL,POWER-SUPPLY-UNIT-FAIL","PowerFail,PowerFail,PowerFail,PowerFail"
14922559.0,"48V-FAIL,FAN-FAIL","PowerFail,PowerFail,PowerFail,PowerFail,PowerF..."
14999487.0,"OPTICAL-LOSS-OF-SIGNAL,OPTICAL-REMOVE","UnitFail,UnitFail"


In [ ]:
train_site['root_cause_type'] = train_site['root_cause_type'].apply(replace_root_cause_type)
train_site.head()

,alarmmsg_original,root_cause_type
ticketno,,
14753084.0,"ETHERNET-LINK-FAIL,ETHERNET-NO-RECEIVE-TRAFFIC...",PowerFail
14771766.0,"ETHERNET-LINK-FAIL,POWER-SUPPLY-UNIT-FAIL",PowerFail
14777089.0,"ETHERNET-LINK-FAIL,POWER-SUPPLY-UNIT-FAIL",PowerFail
14790052.0,POWER-SUPPLY-UNIT-FAIL,PowerFail
14879922.0,"ETHERNET-LINK-FAIL,ETHERNET-NO-RECEIVE-TRAFFIC...",PowerFail


In [ ]:
train_site.reset_index(inplace = True)

## **test**

In [ ]:
# ticketno를 그룹화하여 level과 msg 리스트로 만들기

test_site = pd.DataFrame(test_new.groupby(['ticketno'])[['alarmmsg_original']].agg(list))
test_site.head()

,alarmmsg_original
ticketno,
15238899.0,"[ETHERNET-LINK-FAIL,POWER-SUPPLY-UNIT-FAIL]"
15712444.0,[POWER-SUPPLY-UNIT-FAIL]
15723187.0,"[ETHERNET-NO-RECEIVE-TRAFFIC,POWER-SUPPLY-UNIT..."
15737103.0,"[ETHERNET-LINK-FAIL,POWER-SUPPLY-UNIT-FAIL]"
15737132.0,"[ETHERNET-LINK-FAIL,POWER-SUPPLY-UNIT-FAIL]"


In [ ]:
max_count_ticketno = max(test_site['alarmmsg_original'].apply(len))
max_count_ticketno

1

In [ ]:
# alarmmsg_original 변경 값이 없는지 확인
different_value_count(test_site, 'alarmmsg_original')

0


In [ ]:
# map(lambda)를 사용하여 리스트 풀기
test_site = pd.DataFrame(test_site.applymap(lambda x: unpack_list(x)))

print(len(test_site))
test_site.head(10)

4327


,alarmmsg_original
ticketno,
15238899.0,"ETHERNET-LINK-FAIL,POWER-SUPPLY-UNIT-FAIL"
15712444.0,POWER-SUPPLY-UNIT-FAIL
15723187.0,"ETHERNET-NO-RECEIVE-TRAFFIC,POWER-SUPPLY-UNIT-..."
15737103.0,"ETHERNET-LINK-FAIL,POWER-SUPPLY-UNIT-FAIL"
15737132.0,"ETHERNET-LINK-FAIL,POWER-SUPPLY-UNIT-FAIL"
16188560.0,BOOTING
16199806.0,POWER-SUPPLY-UNIT-FAIL
16201784.0,"ETHERNET-LINK-FAIL,ETHERNET-NO-RECEIVE-TRAFFIC..."
16203726.0,"ETHERNET-LINK-FAIL,ETHERNET-NO-RECEIVE-TRAFFIC..."


In [ ]:
test_site.reset_index(inplace = True)

# **transformer를 위해 만든 msg 조합**

In [ ]:
train_msg = train_site.copy()
train_msg.head()

,ticketno,alarmmsg_original,root_cause_type
0,14753084.0,"ETHERNET-LINK-FAIL,ETHERNET-NO-RECEIVE-TRAFFIC...",PowerFail
1,14771766.0,"ETHERNET-LINK-FAIL,POWER-SUPPLY-UNIT-FAIL",PowerFail
2,14777089.0,"ETHERNET-LINK-FAIL,POWER-SUPPLY-UNIT-FAIL",PowerFail
3,14790052.0,POWER-SUPPLY-UNIT-FAIL,PowerFail
4,14879922.0,"ETHERNET-LINK-FAIL,ETHERNET-NO-RECEIVE-TRAFFIC...",PowerFail


In [ ]:
test_msg = test_site.copy()
test_msg.head()

,ticketno,alarmmsg_original
0,15238899.0,"ETHERNET-LINK-FAIL,POWER-SUPPLY-UNIT-FAIL"
1,15712444.0,POWER-SUPPLY-UNIT-FAIL
2,15723187.0,"ETHERNET-NO-RECEIVE-TRAFFIC,POWER-SUPPLY-UNIT-..."
3,15737103.0,"ETHERNET-LINK-FAIL,POWER-SUPPLY-UNIT-FAIL"
4,15737132.0,"ETHERNET-LINK-FAIL,POWER-SUPPLY-UNIT-FAIL"


In [ ]:
print('Shape of train : {}'.format(train_msg.shape))
print('Shape of test : {}'.format(test_msg.shape))

Shape of train : (1114, 3)
Shape of test : (4327, 2)


In [ ]:
def unpack_list(lst):
    """
    list를 str로 변환하는 함수

    Args:
        lst (list): str로 변환하고 싶은 list

    Returns:
        str로 변환된 list의 요소들을 ','로 연결시킨 문자열
    """

    return ','.join(str(x) for x in lst)

In [ ]:
def replace_root_cause_type(row):
    if 'PowerFail' in row:
        return 'PowerFail'
    elif 'UnitFail' in row:
        return 'UnitFail'
    elif 'LinkCut' in row:
        return 'LinkCut'
    return row

# **tokenized**

In [ ]:
# ' ' 단위로 alarmmsg를 split

train_msg['alarmmsg_original'] = train_msg['alarmmsg_original'].str.replace('-', ' ')
train_msg['alarmmsg_original'] = train_msg['alarmmsg_original'].apply(lambda x: x.split(','))
train_msg['alarmmsg_original'] = train_msg['alarmmsg_original'].apply(lambda x: ' '.join(x))
train_msg.head()

,ticketno,alarmmsg_original,root_cause_type
0,14753084.0,ETHERNET LINK FAIL ETHERNET NO RECEIVE TRAFFIC...,PowerFail
1,14771766.0,ETHERNET LINK FAIL POWER SUPPLY UNIT FAIL,PowerFail
2,14777089.0,ETHERNET LINK FAIL POWER SUPPLY UNIT FAIL,PowerFail
3,14790052.0,POWER SUPPLY UNIT FAIL,PowerFail
4,14879922.0,ETHERNET LINK FAIL ETHERNET NO RECEIVE TRAFFIC...,PowerFail


In [ ]:
# 1. Tokenization

texts = train_msg['alarmmsg_original'].tolist()

tokenized_texts = [text.split() for text in texts]

In [ ]:
# 단어를 고유한 정수로 매핑하는 사전 생성
# 중복된 단어를 처리하기 위해 Counter를 사용해서 정수 index 할당
# Padding, Unknown 추가

from collections import Counter

word_counts = Counter(word for text in tokenized_texts for word in text)
word_to_index = {"[PAD]": 0, "[UNK]": 1}
for word in word_counts:
    if word not in word_to_index:
        word_to_index[word] = len(word_to_index)

In [ ]:
# 단어를 정수로 매핑된 값으로 변환

mapped_texts = [[word_to_index[word] for word in text] for text in tokenized_texts]

In [ ]:
# 1114와 동일

len(mapped_texts)

1114

In [ ]:
# test length 확인 후, 더 큰 값으로 변경 필요
# test length가 더 크면 max_length를 변경 후, 다시 padding

# max_length = max(len(text) for text in mapped_texts)
max_length = 41

padded_texts = [text + [0] * (max_length - len(text)) for text in mapped_texts]

In [ ]:
max_length

41

In [ ]:
len(padded_texts)

1114

In [ ]:
# 5개의 결과 확인
for i, text in enumerate(padded_texts[:5]):
    print('Original Text: {}'.format(texts[i]))
    print('Mapped Text: {}'.format(text))
    print('-' * 30)

Original Text: ETHERNET LINK FAIL ETHERNET NO RECEIVE TRAFFIC POWER SUPPLY UNIT FAIL
Mapped Text: [2, 3, 4, 2, 5, 6, 7, 8, 9, 10, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
------------------------------
Original Text: ETHERNET LINK FAIL POWER SUPPLY UNIT FAIL
Mapped Text: [2, 3, 4, 8, 9, 10, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
------------------------------
Original Text: ETHERNET LINK FAIL POWER SUPPLY UNIT FAIL
Mapped Text: [2, 3, 4, 8, 9, 10, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
------------------------------
Original Text: POWER SUPPLY UNIT FAIL
Mapped Text: [8, 9, 10, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
------------------------------
Original Text: ETHERNET LINK FAIL ETHERNET NO RECEIVE TRAFFIC POWER SUPPLY UNIT FAIL
Ma

In [ ]:
# 단어와 정수 index가 매핑된 dict 생성

text_dict = {texts[i]: text for i, text in enumerate(padded_texts)}

# **test_tokenized**

In [ ]:
test_msg = test_site.copy()

In [ ]:
test_msg['alarmmsg_original'] = test_msg['alarmmsg_original'].str.replace('-', ' ')
test_msg['alarmmsg_original'] = test_msg['alarmmsg_original'].apply(lambda x: x.split(','))
test_msg['alarmmsg_original'] = test_msg['alarmmsg_original'].apply(lambda x: ' '.join(x))
test_msg.head()

,ticketno,alarmmsg_original
0,15238899.0,ETHERNET LINK FAIL POWER SUPPLY UNIT FAIL
1,15712444.0,POWER SUPPLY UNIT FAIL
2,15723187.0,ETHERNET NO RECEIVE TRAFFIC POWER SUPPLY UNIT ...
3,15737103.0,ETHERNET LINK FAIL POWER SUPPLY UNIT FAIL
4,15737132.0,ETHERNET LINK FAIL POWER SUPPLY UNIT FAIL


In [ ]:
texts_test = test_msg['alarmmsg_original'].tolist()

tokenized_texts_test = [text.split() for text in texts_test]

In [ ]:
# 새로운 단어를 word_to_index 매핑에 추가

for sentence_tokens in tokenized_texts_test:
    for word in sentence_tokens:
        if word not in word_to_index:
            word_to_index[word] = len(word_to_index)

print('Updated word_to_index: \n', word_to_index)

Updated word_to_index: 
 {'[PAD]': 0, '[UNK]': 1, 'ETHERNET': 2, 'LINK': 3, 'FAIL': 4, 'NO': 5, 'RECEIVE': 6, 'TRAFFIC': 7, 'POWER': 8, 'SUPPLY': 9, 'UNIT': 10, '48V': 11, 'FAN': 12, 'OPTICAL': 13, 'LOSS': 14, 'OF': 15, 'SIGNAL': 16, 'REMOVE': 17, 'NON': 18, 'VOLATILE': 19, 'RANDOM': 20, 'ACCESS': 21, 'MEMORY': 22, 'BATTERY': 23, 'LOW': 24, 'DATA': 25, 'COMMUNICATE': 26, 'CHANNEL': 27, 'TRANSMISSION': 28, 'PROTECTION': 29, 'SWITCHED': 30, 'LABEL': 31, 'PATH': 32, 'CONNECT': 33, 'AUTO': 34, 'NEGOTIATION': 35, 'MISMATCH': 36, 'BOOTING': 37, 'OPERATING': 38, 'SYSTEM': 39, 'RESTARTED': 40, 'PORT': 41, 'SHUTDOWN': 42, 'TIME': 43, 'DIVISION': 44, 'MULTIPLEX': 45, 'PSEUDOWIRE': 46, 'PACKET': 47, 'INTEGRATED': 48, 'CONTROL': 49, 'TRACK': 50, 'ERROR': 51, 'REMOTE': 52, 'QUALITY': 53, 'MANUAL': 54, 'ALARM': 55, 'MAINTENANCE': 56, 'END': 57, 'POINT': 58, 'HIGH': 59, 'CLIENT': 60, 'DROP': 61, '1': 62, 'DAY': 63, 'EMULATION': 64, 'SERVER': 65, '15': 66, 'MINUTE': 67, 'ENVIRONMENT': 68, 'INPUT': 69,

In [ ]:
mapped_texts_test = []
for sentence_tokens in tokenized_texts_test:
    sentence_indices = [word_to_index[word] if word in word_to_index else word_to_index['[UNK]'] for word in sentence_tokens]
    mapped_texts_test.append(sentence_indices)

In [ ]:
# 4327과 동일

len(mapped_texts_test)

4327

In [ ]:
# max_length_test = 41
max_length_test = max(len(text) for text in mapped_texts_test)

padded_texts_test = [text + [0] * (max_length_test - len(text)) for text in mapped_texts_test]

In [ ]:
max_length_test

41

In [ ]:
len(padded_texts_test)

4327

In [ ]:
for i, text in enumerate(padded_texts_test[:5]):
    print('Original Text: {}'.format(texts_test[i]))
    print('Mapped Text: {}'.format(text))
    print('-' * 30)

Original Text: ETHERNET LINK FAIL POWER SUPPLY UNIT FAIL
Mapped Text: [2, 3, 4, 8, 9, 10, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
------------------------------
Original Text: POWER SUPPLY UNIT FAIL
Mapped Text: [8, 9, 10, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
------------------------------
Original Text: ETHERNET NO RECEIVE TRAFFIC POWER SUPPLY UNIT FAIL
Mapped Text: [2, 5, 6, 7, 8, 9, 10, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
------------------------------
Original Text: ETHERNET LINK FAIL POWER SUPPLY UNIT FAIL
Mapped Text: [2, 3, 4, 8, 9, 10, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
------------------------------
Original Text: ETHERNET LINK FAIL POWER SUPPLY UNIT FAIL
Mapped Text: [2, 3, 4, 8, 9, 10, 4, 0, 0, 0, 0, 0

# **Modeling**

In [ ]:
print(len(word_to_index))
print(len(text_dict))

124
91


In [ ]:
# embedding matrix 생성


# vocab_size = embedding layer에 들어갈 수 있는 단어집합의 크기
# 위에서의 padding = 시퀀스 내의 패딩 : 시퀀스 데이에서 각 시퀀스의 길이를 맞추기 위해 빈 값을 추가하는 것
# 여기서의 padding = 임베딩 행렬의 패딩 : 단어 집합에 속하지 않는 단어에 해당하는 벡터를 행렬에 추가하는 것

# 단어 개수 + 1 (패딩을 위한 0)
vocab_size = len(word_to_index) + 1
embedding_dim = 100

np.random.seed(seed_value)
embedding_matrix = np.random.random((vocab_size, embedding_dim))

In [ ]:
# 125
vocab_size

125

In [ ]:
embedding_matrix

array([[0.15266373, 0.30235661, 0.06203641, ..., 0.2025168 , 0.67985305,
        0.46534506],
       [0.92001748, 0.72820735, 0.24585653, ..., 0.4459065 , 0.76829829,
        0.16288936],
       [0.77254856, 0.53478201, 0.15692983, ..., 0.00623583, 0.70304607,
        0.1365286 ],
       ...,
       [0.02726336, 0.47426486, 0.27277894, ..., 0.30433057, 0.17459116,
        0.34618255],
       [0.34972011, 0.13641084, 0.78393794, ..., 0.35627762, 0.79515618,
        0.36814218],
       [0.26734824, 0.68311401, 0.97262003, ..., 0.72763245, 0.42860207,
        0.49133067]])

In [ ]:
def positional_encoding(position, d_model, seed_value):
    np.random.seed(seed_value)

    angle_rads = np.arange(position)[:, np.newaxis] / np.power(10000, (2 * (np.arange(d_model)[np.newaxis, :] // 2)) / np.float32(d_model))
    # 짝수 인덱스에는 sin 함수 적용
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    # 홀수 인덱스에는 cos 함수 적용
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])
    pos_encoding = angle_rads[np.newaxis, ...]
    return pos_encoding

# 가장 긴 문장의 길이
max_length = max_length_test
# 임베딩 차원
d_model = 100

# 포지셔널 인코딩 생성
pos_encoding = positional_encoding(max_length, d_model, seed_value)
pos_encoding

array([[[ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00, ...,
          1.00000000e+00,  0.00000000e+00,  1.00000000e+00],
        [ 8.41470985e-01,  5.40302306e-01,  7.39120549e-01, ...,
          9.99999990e-01,  1.20226443e-04,  9.99999993e-01],
        [ 9.09297427e-01, -4.16146837e-01,  9.95703541e-01, ...,
          9.99999958e-01,  2.40452885e-04,  9.99999971e-01],
        ...,
        [ 2.96368579e-01,  9.55073644e-01,  1.89935810e-01, ...,
          9.99984915e-01,  4.56858896e-03,  9.99989564e-01],
        [ 9.63795386e-01,  2.66642932e-01,  8.53601641e-01, ...,
          9.99984111e-01,  4.68881411e-03,  9.99989007e-01],
        [ 7.45113160e-01, -6.66938062e-01,  9.59990515e-01, ...,
          9.99983286e-01,  4.80903920e-03,  9.99988437e-01]]])

In [ ]:
pos_encoding.shape

(1, 41, 100)

In [ ]:
# embedded_inputs = Embedding(input_dim = vocab_size, output_dim = embedding_dim)(text_dict)

embedded_text = [text_dict[text] for text in texts]
embedded_text = tf.convert_to_tensor(embedded_text)
embedded_inputs = Embedding(input_dim = vocab_size, output_dim = 100)(embedded_text)

position = max_length
pos_encoding = positional_encoding(position, embedding_dim, seed_value)

inputs = embedded_inputs + pos_encoding

In [ ]:
# tf.Tensor: shape = (1114, 41, 100)
inputs

<tf.Tensor: shape=(1114, 41, 100), dtype=float32, numpy=
array([[[ 0.04543222,  1.0171981 , -0.01041202, ...,  1.0034335 ,
          0.02624745,  1.0250744 ],
        [ 0.8024617 ,  0.58812684,  0.7075701 , ...,  0.98969966,
          0.01751875,  0.96961224],
        [ 0.85974383, -0.3840509 ,  1.0237013 , ...,  0.96572655,
         -0.0179674 ,  1.0091624 ],
        ...,
        [ 0.27214122,  0.9220673 ,  0.17092215, ...,  0.9968686 ,
          0.05172287,  0.99975485],
        [ 0.93956804,  0.23363657,  0.834588  , ...,  0.9968678 ,
          0.0518431 ,  0.9997543 ],
        [ 0.7208858 , -0.69994444,  0.94097686, ...,  0.996867  ,
          0.05196332,  0.9997537 ]],

       [[ 0.04543222,  1.0171981 , -0.01041202, ...,  1.0034335 ,
          0.02624745,  1.0250744 ],
        [ 0.8024617 ,  0.58812684,  0.7075701 , ...,  0.98969966,
          0.01751875,  0.96961224],
        [ 0.85974383, -0.3840509 ,  1.0237013 , ...,  0.96572655,
         -0.0179674 ,  1.0091624 ],
        ..

In [ ]:
# Transformer Block 구성
# seed 고정을 위해 MultiHeadAttention layer에 dropout = 0 부여

def transformer_block(inputs, embedding_dim, num_heads, ff_dim, seed_value, rate = 0.1):
    attn_output = MultiHeadAttention(num_heads = num_heads, key_dim = embedding_dim, dropout = 0)(inputs, inputs, training = True)
    attn_output = Dropout(rate, seed = seed_value)(attn_output, training = True)
    out1 = LayerNormalization(epsilon = 1e-6)(inputs + attn_output)
    ffn_output = Dense(ff_dim, activation = 'relu')(out1)
    ffn_output = Dense(embedding_dim)(ffn_output)
    ffn_output = Dropout(rate, seed = seed_value)(ffn_output)
    out2 = LayerNormalization(epsilon = 1e-6)(out1 + ffn_output)
    return out2

In [ ]:
def build_transformer_model(embedding_dim, num_heads, ff_dim, seed_value, num_classes, num_transformer_blocks, input_shape):
    inputs = Input(shape = input_shape)

    # Embedding Layer
    embedded_inputs = Embedding(input_dim = vocab_size, output_dim = embedding_dim)(inputs)

    # Positional Encoding
    position = max_length
    pos_encoding = positional_encoding(position, embedding_dim, seed_value)

    # 최종 input
    inputs_with_position = embedded_inputs + pos_encoding

    # Transformer Blocks
    x = inputs_with_position
    for _ in range(num_transformer_blocks):
        x = transformer_block(x, embedding_dim, num_heads, ff_dim, seed_value)

    # Global Average Pooling & Classification Layer
    x = GlobalAveragePooling1D()(x)
    outputs = Dense(num_classes, activation = 'softmax')(x)

    model = Model(inputs = inputs, outputs = outputs)
    return model

In [ ]:
# 모델 생성
model = build_transformer_model(
    embedding_dim = 100,
    num_heads = 4,
    ff_dim = 64,
    seed_value = seed_value,
    num_classes = 3,
    num_transformer_blocks = 2,
    input_shape = (max_length, )
)

In [ ]:
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 41)]         0           []                               
                                                                                                  
 embedding_9 (Embedding)        (None, 41, 100)      12500       ['input_6[0][0]']                
                                                                                                  
 tf.__operators__.add_25 (TFOpL  (None, 41, 100)     0           ['embedding_9[0][0]']            
 ambda)                                                                                           
                                                                                                  
 multi_head_attention_10 (Multi  (None, 41, 100)     161300      ['tf.__operators__.add_25[0

In [ ]:
model.compile('adam', 'categorical_crossentropy', metrics = ['accuracy'])

# **model.fit**

In [ ]:
y = train_msg['root_cause_type']
y

0       PowerFail
1       PowerFail
2       PowerFail
3       PowerFail
4       PowerFail
          ...    
1109      LinkCut
1110      LinkCut
1111      LinkCut
1112      LinkCut
1113      LinkCut
Name: root_cause_type, Length: 1114, dtype: object

In [ ]:
y = np.array(y).reshape(-1, 1)
y

array([['PowerFail'],
       ['PowerFail'],
       ['PowerFail'],
       ...,
       ['LinkCut'],
       ['LinkCut'],
       ['LinkCut']], dtype=object)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(padded_texts, y, test_size = 0.1, stratify = y, random_state = seed_value)

In [ ]:
# target encoding

ohe = OneHotEncoder(sparse = False)
y_train_encoded = ohe.fit_transform(y_train)
y_test_encoded = ohe.transform(y_test)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
y_train_encoded = y_train_encoded.astype(int)
y_test_encoded = y_test_encoded.astype(int)

In [ ]:
type(y_train_encoded)

numpy.ndarray

In [ ]:
X_train = np.array(X_train)
X_test = np.array(X_test)

In [ ]:
print('type of y :', type(y_train_encoded))
print('type of X :', type(X_train))

type of y : <class 'numpy.ndarray'>
type of X : <class 'numpy.ndarray'>


In [ ]:
print('X_train shape:', len(X_train))
print('y_train_encoded shape:', y_train_encoded.shape)
print('X_test shape:', len(X_test))
print('y_test_encoded shape:', y_test_encoded.shape)

X_train shape: 1002
y_train_encoded shape: (1002, 3)
X_test shape: 112
y_test_encoded shape: (112, 3)


In [ ]:
history = model.fit(X_train, y_train_encoded, batch_size = 8, epochs = 100, validation_data = (X_test, y_test_encoded))

Epoch 1/100
126/126 [==============================] - 14s 62ms/step - loss: 1.0470 - accuracy: 0.5150 - val_loss: 0.6277 - val_accuracy: 0.6696
Epoch 2/100
126/126 [==============================] - 9s 70ms/step - loss: 0.3046 - accuracy: 0.9032 - val_loss: 0.2128 - val_accuracy: 0.9554
Epoch 3/100
126/126 [==============================] - 7s 53ms/step - loss: 0.1435 - accuracy: 0.9541 - val_loss: 0.1512 - val_accuracy: 0.9554
Epoch 4/100
126/126 [==============================] - 9s 69ms/step - loss: 0.1089 - accuracy: 0.9601 - val_loss: 0.1111 - val_accuracy: 0.9821
Epoch 5/100
126/126 [==============================] - 7s 52ms/step - loss: 0.0819 - accuracy: 0.9641 - val_loss: 0.0967 - val_accuracy: 0.9821
Epoch 6/100
126/126 [==============================] - 9s 70ms/step - loss: 0.0772 - accuracy: 0.9731 - val_loss: 0.0750 - val_accuracy: 0.9821
Epoch 7/100
126/126 [==============================] - 7s 55ms/step - loss: 0.0812 - accuracy: 0.9750 - val_loss: 0.2755 - val_accuracy

# **predict**

In [ ]:
test_input = np.array(padded_texts_test)

In [ ]:
predict = model.predict(test_input)

136/136 [==============================] - 9s 61ms/step


In [ ]:
predict

array([[4.84083066e-05, 9.99847889e-01, 1.03810387e-04],
       [4.68389117e-05, 9.99838352e-01, 1.14837945e-04],
       [7.03133061e-04, 9.99288797e-01, 7.94641073e-06],
       ...,
       [9.99864936e-01, 1.02084930e-04, 3.29834911e-05],
       [9.99857426e-01, 1.11915717e-04, 3.06572147e-05],
       [9.99811232e-01, 6.30488794e-05, 1.25757826e-04]], dtype=float32)

In [ ]:
# 각 샘플의 클래스 인덱스 추출
predicted_class_index = np.argmax(predict, axis = 1)

predicted_labels = []
for idx in predicted_class_index:
    if idx == 0:
        predicted_labels.append('LinkCut')
    elif idx == 1:
        predicted_labels.append('PowerFail')
    else:
        predicted_labels.append('UnitFail')

predicted_labels

['PowerFail',
 'PowerFail',
 'PowerFail',
 'PowerFail',
 'PowerFail',
 'UnitFail',
 'PowerFail',
 'PowerFail',
 'PowerFail',
 'UnitFail',
 'PowerFail',
 'UnitFail',
 'UnitFail',
 'PowerFail',
 'PowerFail',
 'UnitFail',
 'UnitFail',
 'UnitFail',
 'UnitFail',
 'UnitFail',
 'PowerFail',
 'PowerFail',
 'PowerFail',
 'PowerFail',
 'PowerFail',
 'PowerFail',
 'UnitFail',
 'UnitFail',
 'UnitFail',
 'UnitFail',
 'UnitFail',
 'UnitFail',
 'UnitFail',
 'UnitFail',
 'UnitFail',
 'UnitFail',
 'UnitFail',
 'UnitFail',
 'UnitFail',
 'UnitFail',
 'UnitFail',
 'UnitFail',
 'UnitFail',
 'UnitFail',
 'UnitFail',
 'UnitFail',
 'UnitFail',
 'UnitFail',
 'UnitFail',
 'UnitFail',
 'UnitFail',
 'UnitFail',
 'UnitFail',
 'UnitFail',
 'UnitFail',
 'UnitFail',
 'UnitFail',
 'UnitFail',
 'UnitFail',
 'UnitFail',
 'UnitFail',
 'UnitFail',
 'UnitFail',
 'UnitFail',
 'UnitFail',
 'UnitFail',
 'UnitFail',
 'UnitFail',
 'UnitFail',
 'UnitFail',
 'UnitFail',
 'UnitFail',
 'UnitFail',
 'UnitFail',
 'UnitFail',
 'UnitFa

In [ ]:
label_sample = label_sample_org.copy()
label_sample['root_cause_type'] = predicted_labels

In [ ]:
label_sample['root_cause_type'].value_counts() / len(label_sample)

LinkCut      0.648024
PowerFail    0.311994
UnitFail     0.039982
Name: root_cause_type, dtype: float64